# 3주차 Day3 — RAG 파이프라인 실습

> **오후 실습용** | 수강생 직접 실행 노트북
>
> 오전에 데모로 본 RAG 시스템을 **직접 만들어봅니다.**

---

## 실습 구성

| 블록 | Section | 내용 | 시간 |
|------|---------|------|------|
| **오후1** (13:30~14:20) | Section 0~2 | 환경설정 + 벡터DB + 검색 테스트 | 10분 |
| | **Section 3** | **RetrievalQA 체인 구축 + 테스트 ** | 25분 |
| | 체크포인트 | 4개 항목 통과 확인 | 10분 |
| **오후2** (14:30~15:20) | Section 4 | 커스텀 프롬프트 적용 | 20분 |
| | Section 5 | 출처 표시 함수 구현 | 20분 |
| | Section 6 | 연습 과제 | 10분 |

---

### 학습 목표
1. PDF 문서 기반 RAG 파이프라인을 end-to-end로 구축할 수 있다
2. 출처(파일명 + 페이지)를 포함한 신뢰할 수 있는 응답을 생성할 수 있다
3. 커스텀 프롬프트로 답변 품질을 개선할 수 있다

---
---
# Section 0: 환경설정

> **2분** | 점심에 미리 실행하셨다면 바로 다음 섹션으로!
>
> 이 셀을 실행하면 필요한 패키지가 설치됩니다.

In [ ]:
# ============================================================
# Section 0: 패키지 설치
# RAG 파이프라인에 필요한 라이브러리를 한 번에 설치합니다.
# -q 옵션: 설치 로그를 간략하게 표시
# ============================================================

!pip install -q langchain langchain-openai langchain-community chromadb pypdf tiktoken
# langchain: LLM 체인 프레임워크 (2주차에 학습)
# langchain-openai: OpenAI 모델 연결
# langchain-community: PDF 로더, ChromaDB 연결 등
# chromadb: 벡터 데이터베이스 (Day1~2에 학습)
# pypdf: PDF 파일 읽기
# tiktoken: 토큰 수 계산 (OpenAI 내부 사용)


In [ ]:
# ============================================================
# API 키 설정
# OpenAI 서비스를 사용하려면 인증키가 필요합니다.
# 제공받은 키를 아래 "sk-..." 부분에 붙여넣으세요.
# ============================================================

import os  # 운영체제 환경변수를 다루는 파이썬 기본 모듈

os.environ["OPENAI_API_KEY"] = "sk-..."  # ← 여기에 제공받은 키 입력

# 키가 입력됐는지 자동 확인 ("sk-..." 그대로면 에러 발생)
assert os.environ.get("OPENAI_API_KEY") != "sk-...", "API 키를 입력해주세요!"
print("API 키 설정 완료")


---
# Section 1: 벡터DB 구축

> **3분** | Day2에서 이미 해본 내용입니다. 셀 3개를 순서대로 실행하세요.
>
> 결과 숫자만 확인하면 됩니다.

In [ ]:
# ============================================================
# Step 1: PDF 로딩
# 사양서(4페이지) + 시험성적서(3페이지) = 총 7페이지를 읽어옵니다
# ============================================================

# PyPDFLoader: PDF 파일을 읽어서 페이지별로 텍스트를 추출하는 도구
from langchain_community.document_loaders import PyPDFLoader

# 읽어올 PDF 파일 경로 목록
pdf_files = [
    "data/제품사양서_스마트냉장고_RF9000.pdf",
    "data/시험성적서_스마트냉장고_RF9000.pdf"
]

all_docs = []                        # 모든 페이지를 모아둘 빈 리스트
for pdf_path in pdf_files:           # PDF 파일을 하나씩 순회
    loader = PyPDFLoader(pdf_path)    # PDF 읽기 도구 생성
    docs = loader.load()             # 실제로 PDF를 읽어서 페이지별로 분리
    all_docs.extend(docs)            # 전체 목록에 추가
    print(f"{pdf_path.split('/')[-1]} → {len(docs)}페이지 로딩")

print(f"\n총 {len(all_docs)}페이지 로딩 완료")  # ← 7이 나와야 합니다


In [ ]:
# ============================================================
# Step 2: 청킹 (문서를 작은 조각으로 분할)
# 한 페이지가 너무 길면 AI가 핵심을 못 찾습니다.
# 500자씩 잘라서 검색 정확도를 높이는 과정입니다.
# ============================================================

from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,    # 한 조각의 최대 글자 수 (A4 반 페이지 정도)
    chunk_overlap=50   # 조각 간 50자 겹침 (문맥이 끊기지 않도록)
)
chunks = splitter.split_documents(all_docs)  # 7페이지 → 여러 조각으로 분할

print(f"{len(chunks)}개 청크로 분할")  # ← 15~20개 범위면 정상


In [ ]:
# ============================================================
# Step 3: 벡터DB 저장
# 각 조각을 숫자 벡터로 변환(임베딩)해서 ChromaDB에 저장합니다.
# 이후 질문하면 "의미가 비슷한 조각"을 빠르게 찾을 수 있습니다.
# ============================================================

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()  # 텍스트 → 숫자 벡터 변환기 (Day1 학습)
vectorstore = Chroma.from_documents(
    chunks,      # 위에서 만든 문서 조각들
    embeddings   # 임베딩 변환기
)  # 조각들을 벡터로 변환 후 DB에 저장

print(f"벡터DB 구축 완료 ({len(chunks)}개 청크 저장)")


---
# Section 2: 유사도 검색 테스트

> **2분** | Day2에서 배운 유사도 검색이 잘 되는지 빠르게 확인합니다.
>
> 벡터DB가 정상 구축됐는지 검증하는 단계입니다.

In [ ]:
# ============================================================
# Section 2: 유사도 검색 테스트 (Day2 복습)
# 벡터DB에서 질문과 의미가 비슷한 문서 조각 3개를 찾아봅니다.
# 이 검색이 RAG의 핵심 단계입니다. (5단계 중 Step 2)
# ============================================================

query = "RF9000의 총 용량은?"  # 테스트 질문

# similarity_search: 질문과 의미가 비슷한 조각을 k개 찾는 함수
results = vectorstore.similarity_search(query, k=3)  # 가장 유사한 3개 검색

print(f"검색 결과: {len(results)}개 문서 조각 찾음")
print()
for i, doc in enumerate(results):  # enumerate: 순번(i)과 내용(doc)을 함께 꺼냄
    src = doc.metadata.get('source', '').split('/')[-1]  # 파일명 추출
    pg = doc.metadata.get('page', 0) + 1                 # 페이지 번호 (0→1 변환)
    print(f"[{i+1}] {src} (p.{pg})")
    print(f"  {doc.page_content[:80]}...")  # 조각 내용 앞 80자만 미리보기
    print()


> 검색 결과가 3개 나오고, 제품사양서의 용량 관련 내용이 보이면 정상입니다.
>
> 이 검색 결과가 다음 Section 3에서 RetrievalQA 체인의 **retriever**가 자동으로 가져오는 것과 같습니다.

---
---
# Section 3: RetrievalQA 체인 구축

> **25분** | **오늘의 핵심!** 오전2에서 배운 코드를 직접 타이핑합니다.
>
> 오전에 슬라이드에서 본 5줄 코드, 기억나시죠?
>
> 이제 직접 만들어봅시다.


In [ ]:
# ============================================================
# RetrievalQA 체인 생성 — 오늘의 핵심 코드!
#
# 이 코드 한 블록이 오전2에서 배운 5단계를 전부 수행합니다:
#   llm                    → Step 4: 답변 생성에 사용할 AI 모델
#   chain_type             → Step 3: 문서와 질문을 조합하는 방식
#   retriever              → Step 1~2: 질문을 벡터로 변환 + 유사 문서 검색
#   return_source_documents → Step 5: 어떤 문서를 참고했는지 출처 반환
# ============================================================

from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),  # AI 모델 (temperature=0: 정확하게)
    chain_type="stuff",                                   # 검색된 조각을 그대로 프롬프트에 넣기
    retriever=vectorstore.as_retriever(                   # 벡터DB를 검색기로 변환
        search_kwargs={"k": 3}                           # 가장 비슷한 조각 3개 검색
    ),
    return_source_documents=True  # 출처 정보도 함께 반환 (True가 핵심!)
)

print("RetrievalQA 체인 생성 완료!")


### 첫 질의 실행!

> 코드를 실행하고 첫 번째 질문을 던져보세요.

In [ ]:
# ============================================================
# 첫 번째 질문!
# qa.invoke()가 내부적으로 5단계를 전부 실행합니다.
# ============================================================

result = qa.invoke({"query": "RF9000의 총 용량은?"})  # 질문 실행!

# result는 딕셔너리: {"result": "답변 텍스트", "source_documents": [참고 문서들]}
print("답변:")
print(result["result"])       # AI가 생성한 답변

print("\n출처:")
for doc in result["source_documents"]:   # 참고한 문서 조각 목록
    src = doc.metadata.get("source", "").split("/")[-1]  # 파일명만 추출
    pg = doc.metadata.get("page", 0) + 1                 # 0→1 페이지 변환
    print(f"- {src} (p.{pg})")


> **답변이 나왔다면, 축하합니다! 여러분의 RAG가 동작합니다!**
>
> 예상 결과: RF9000의 총 용량은 868L (냉장 524L / 냉동 344L)
>
> 출처: 제품사양서_스마트냉장고_RF9000.pdf (p.2)

### 질의 5개 테스트

> 아래 질문을 하나씩 실행하고 답변을 확인하세요.
>
> 각 질문마다 확인할 것:
> 1. 답변이 정확한가?
> 2. 출처(source_documents)가 나오는가?
> 3. 어떤 PDF의 몇 페이지인가?

In [ ]:
# Q1: 사양서 질문
result = qa.invoke({"query": "RF9000의 총 용량은?"})  # 질문 실행
print("Q1:", result["result"])  # 답변 출력
for doc in result["source_documents"]:  # 참고한 문서 목록 순회
    print(f"  출처: {doc.metadata.get('source','').split('/')[-1]} (p.{doc.metadata.get('page',0)+1})")
#                      │               │              │         │              │              │
#                      │               │              │         │              │              └─ +1: 0부터 시작 → 1부터로 변환 (p.0 → p.1)
#                      │               │              │         │              └─ .get('page', 0): 페이지 번호 꺼냄 (없으면 0)
#                      │               │              │         └─ [-1]: 마지막 조각 = 파일명만
#                      │               │              └─ .split('/'): 경로를 / 기준으로 분리
#                      │               └─ .get('source', ''): source 키 값 꺼냄 (없으면 빈 문자열)
#                      └─ doc.metadata: PDF 로딩 시 자동 생성된 부가정보 딕셔너리
#
# 예시 흐름:
# doc.metadata = {"source": "data/제품사양서_스마트냉장고_RF9000.pdf", "page": 1}
# .get('source','')     → "data/제품사양서_스마트냉장고_RF9000.pdf"
# .split('/')           → ["data", "제품사양서_스마트냉장고_RF9000.pdf"]
# [-1]                  → "제품사양서_스마트냉장고_RF9000.pdf"
# .get('page',0)+1      → 1+1 = 2
# 최종 출력             → "  출처: 제품사양서_스마트냉장고_RF9000.pdf (p.2)"

In [ ]:
# Q2: 사양서 질문 — 사양서 p.2에 답이 있습니다
result = qa.invoke({"query": "소비전력은 얼마인가요?"})  # 질문 실행
print("Q2:", result["result"])  # 답변 출력
for doc in result["source_documents"]:  # 출처 순회
    print(f"  출처: {doc.metadata.get('source','').split('/')[-1]} (p.{doc.metadata.get('page',0)+1})")


In [ ]:
# Q3: 사양서 질문 — 사양서 p.4 에러코드표에 답이 있습니다
result = qa.invoke({"query": "E3 에러코드는 무엇인가요?"})  # 질문 실행
print("Q3:", result["result"])  # 답변 출력
for doc in result["source_documents"]:  # 출처 순회
    print(f"  출처: {doc.metadata.get('source','').split('/')[-1]} (p.{doc.metadata.get('page',0)+1})")


In [ ]:
# Q4: 시험성적서 질문 — 사양서가 아닌 시험성적서에서 답이 나와야 합니다
result = qa.invoke({"query": "에너지효율 등급은?"})  # 질문 실행
print("Q4:", result["result"])  # 답변 출력
for doc in result["source_documents"]:  # 출처 순회
    print(f"  출처: {doc.metadata.get('source','').split('/')[-1]} (p.{doc.metadata.get('page',0)+1})")


In [ ]:
# Q5: 시험성적서 질문 — 시험성적서 p.2 소음 시험 항목에 답이 있습니다
result = qa.invoke({"query": "소음 측정값은 얼마인가요?"})  # 질문 실행
print("Q5:", result["result"])  # 답변 출력
for doc in result["source_documents"]:  # 출처 순회
    print(f"  출처: {doc.metadata.get('source','').split('/')[-1]} (p.{doc.metadata.get('page',0)+1})")


### 오후1 체크포인트 — 4개 항목 통과하기

| # | 항목 | 확인 |
|---|------|------|
| 1 | 패키지 설치 완료 (`pip install` 에러 없이 완료) | |
| 2 | `all_docs = 7개`, `chunks ≈ 15~20개` | |
| 3 | RetrievalQA 답변 + `source_documents` 출력 | |
| 4 | 질의 3개 이상 테스트 완료 (사양서 2개 + 시험성적서 1개 이상) | |

> 4개 다 되셨으면 손 들어주세요!
>
> 쉬는 시간 후 **오후2**: 커스텀 프롬프트 + 출처 표시를 추가합니다.

---
---
# Section 4: 커스텀 프롬프트 — 답변 규칙 추가

> **20분** | 오전 브레인스토밍에서 만든 규칙, 기억나시죠?
>
> 지금 그 규칙을 코드에 직접 넣어봅니다!
>
> **기본 체인 vs 커스텀 체인**의 답변 차이를 직접 확인하세요.

In [ ]:
# ============================================================
# Section 4: 커스텀 프롬프트 — 답변 규칙 추가
#
# 2주차에 배운 PromptTemplate을 RAG에 적용합니다.
# {context} = 벡터DB에서 검색된 문서 조각 (자동 채워짐)
# {question} = 사용자가 입력한 질문 (자동 채워짐)
# ============================================================

from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate(
    template="""다음 문서를 참고하여 질문에 답하세요.

문서 내용: {context}
질문: {question}

답변 규칙:
1. 문서에 있는 정보만 사용하세요
2. 문서에 없으면 "해당 정보를 찾을 수 없습니다"라고 답하세요
3. 수치를 인용할 때는 단위를 반드시 포함하세요

답변:""",
    input_variables=["context", "question"]  # 자동으로 채워질 변수 2개
)

print("커스텀 프롬프트 생성 완료")
print("\n답변 규칙 3가지:")
print("1. 문서에 있는 정보만 사용")
print("2. 없으면 '찾을 수 없습니다'")
print("3. 수치에 단위 포함")


In [ ]:
# ============================================================
# 커스텀 프롬프트를 적용한 RAG 체인
#
# 위의 qa 체인과 거의 동일하지만, 딱 한 줄이 추가됩니다.
# chain_type_kwargs={"prompt": custom_prompt}
# 이 한 줄로 AI의 답변 방식이 완전히 달라집니다!
# ============================================================

custom_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),   # 같은 AI 모델
    chain_type="stuff",                                    # 같은 조립 방식
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),  # 같은 검색기
    return_source_documents=True,                          # 출처 반환
    chain_type_kwargs={"prompt": custom_prompt}            # ← 이 한 줄이 핵심!
)

print("커스텀 RAG 체인 생성 완료")


### 기본 vs 커스텀 비교 테스트

> 같은 질문을 `qa` (기본)과 `custom_qa` (커스텀)에 각각 던져보세요.
>
> 차이가 보이면 성공!

In [ ]:
# 같은 질문으로 기본 vs 커스텀 비교!
query = "절연저항 시험 결과는?"

print("=" * 50)
print("[기본 체인]")
print("=" * 50)
r1 = qa.invoke({"query": query})         # 규칙 없는 기본 체인
print(r1["result"])

print("\n" + "=" * 50)
print("[커스텀 체인]")
print("=" * 50)
r2 = custom_qa.invoke({"query": query})   # 규칙 3가지가 적용된 커스텀 체인
print(r2["result"])


In [ ]:
# 문서에 없는 질문으로 비교! — 할루시네이션 방지 테스트
query = "RF9000 가격은 얼마인가요?"  # 가격은 사양서에도 시험성적서에도 없음

print("=" * 50)
print("[기본 체인] — 가격을 지어낼 수 있음!")
print("=" * 50)
r1 = qa.invoke({"query": query})         # 규칙 없음 → 답변을 지어낼 수 있음
print(r1["result"])

print("\n" + "=" * 50)
print("[커스텀 체인] — '찾을 수 없습니다'로 답해야 정상")
print("=" * 50)
r2 = custom_qa.invoke({"query": query})   # 규칙 2번 → 없으면 "찾을 수 없습니다"
print(r2["result"])


> **차이가 보이셨나요?**
>
> - 기본 체인: 수치나 단위가 빠질 수 있고, 없는 정보를 지어낼 수 있음
> - 커스텀 체인: 규칙 덕분에 정확한 수치+단위, 모르면 "찾을 수 없다"고 답변
>
> **프롬프트 규칙 하나가 이만큼 차이를 만듭니다!**

### 여러분의 규칙 추가!

> 오전 브레인스토밍에서 만든 규칙을 4번에 추가해보세요.
>
> 예시:
> - `4. 표 형식으로 정리해 답변`
> - `4. 안전 수치는 기준값도 포함`
> - `4. 출처 문서명을 답변에 포함`

In [ ]:
# ============================================================
# 여러분의 규칙을 추가해보세요!
# 아래 4번 규칙 부분을 수정한 후 이 셀 전체를 실행하세요.
# 예시: "답변 끝에 관련 에러코드를 함께 안내하세요"
#       "전문 용어는 괄호 안에 쉬운 설명을 추가하세요"
# ============================================================

# 프롬프트 템플릿: AI에게 전달할 지시문 틀
my_prompt = PromptTemplate(
    template="""다음 문서를 참고하여 질문에 답하세요.

문서 내용: {context}
질문: {question}

답변 규칙:
1. 문서에 있는 정보만 사용하세요
2. 문서에 없으면 "해당 정보를 찾을 수 없습니다"라고 답하세요
3. 수치를 인용할 때는 단위를 반드시 포함하세요
4. [여기에 규칙 추가]

답변:""",
    input_variables=["context", "question"]  # 자동으로 채워질 변수
)

# 새 프롬프트로 체인 다시 생성 (구조는 custom_qa와 동일)
my_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),           # AI 모델 선택
    chain_type="stuff",                                            # 문서 조립 방식
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),   # 유사 문서 3개 검색
    return_source_documents=True,                                  # 출처 반환
    chain_type_kwargs={"prompt": my_prompt}                        # ← 나의 프롬프트 적용
)

# 테스트: 규칙이 잘 적용됐는지 확인
result = my_qa.invoke({"query": "E3 에러코드는 무엇인가요?"})  # 질문 실행
print(result["result"])  # 답변 출력


---
---
# Section 5: 출처 표시 함수 구현

> **20분** | 오전3에서 본 `format_response_with_sources()` 함수를 직접 만듭니다.
>
> 질문 + 답변 + 출처를 한 번에 보여주는 함수입니다.

In [ ]:
# ============================================================
# Section 5: 출처 표시 함수
# 질문 → 답변 → 참고 문서를 한 번에 깔끔하게 출력하는 함수입니다.
# 이 함수를 만들어두면 이후 질문할 때 한 줄로 실행할 수 있습니다.
# ============================================================

def ask_with_sources(query):  # query: 질문 텍스트를 받는 매개변수
    """질문하고, 답변과 출처를 함께 출력"""
    result = custom_qa.invoke({"query": query})  # 커스텀 체인으로 질문 실행
    
    print("=" * 50)
    print(f"질문: {query}")
    print("=" * 50)
    print(f"\n답변:\n{result['result']}")       # AI 답변 출력
    
    # 참고한 문서 목록 출력
    print(f"\n참고 문서 ({len(result['source_documents'])}건):")
    for i, doc in enumerate(result['source_documents']):  # 순번과 문서를 함께 순회
        name = doc.metadata.get('source', '').split('/')[-1]  # 파일명 추출
        pg = doc.metadata.get('page', 0) + 1                  # 페이지 번호 (0→1)
        print(f"  [{i+1}] {name} (p.{pg})")
    print("=" * 50)

print("ask_with_sources() 함수 정의 완료")
print('\n사용법: ask_with_sources("질문")')


In [ ]:
# 테스트: 사양서 질문
ask_with_sources("RF9000 냉매 종류는?")  # 함수 하나로 답변 + 출처 한 번에 출력


In [ ]:
# 테스트: 시험성적서 질문 — 출처가 시험성적서로 나오는지 확인
ask_with_sources("에너지효율 등급과 월간 소비전력량은?")


In [ ]:
# 테스트: 교차 참조 — 두 문서에서 답변이 나오는지 확인!
# 출처에 사양서와 시험성적서가 함께 나오면 성공
ask_with_sources("이 제품은 안전한가요?")


> **출처에 두 PDF가 함께 나오면 교차 참조가 된 것입니다!**
>
> 출처를 실제 PDF와 대조해보세요. 페이지 번호가 맞나요?

In [ ]:
# 테스트: 문서에 없는 질문 — "찾을 수 없습니다"로 답해야 정상
ask_with_sources("RF9000 가격은 얼마인가요?")


> 가격은 사양서에도 시험성적서에도 없습니다.
>
> 커스텀 프롬프트 규칙 2번 덕분에 "찾을 수 없습니다"라고 답합니다.
>
> 이것이 **할루시네이션 방지** — 내일 Day4에서 더 깊이 다룹니다.

---
---
# Section 6: 연습 과제

> **10분** | 다 못 끝내도 괜찮습니다. 핵심은 Section 5까지 완료하는 것!

---

## 기본 과제: 질문 5개 + 출처 대조

In [ ]:
# 기본 과제 Q1: 사양서 질문 — 사양서 p.2 주요 사양표에서 확인
ask_with_sources("냉장실 용량은 몇 리터?")


In [ ]:
# 기본 과제 Q2: 사양서 질문 — 사양서 p.4 에러코드 일람에서 확인
ask_with_sources("E3 에러코드는 무엇인가요?")


In [ ]:
# 기본 과제 Q3: 시험성적서 질문 — 시험성적서 p.2 전기안전시험에서 확인
ask_with_sources("절연저항 시험 합격 기준은?")


In [ ]:
# 기본 과제 Q4: 시험성적서 질문 — 시험성적서 p.2 소음 시험에서 확인
ask_with_sources("소음 측정값은?")


In [ ]:
# 기본 과제 Q5: 교차 참조 — 사양서 + 시험성적서 모두 출처에 나오는지 확인
ask_with_sources("이 제품은 안전한가요?")


## 심화 과제: "문서에 없는 질문" 3개

> 커스텀 프롬프트 덕분에 "해당 정보를 찾을 수 없습니다"라고 답해야 정상!
>
> → 잘못된 출처를 발견하면 메모해두세요! 내일(Day4) 할루시네이션 대응에서 다시 다룹니다.

In [ ]:
# 심화 Q1: 가격 — 사양서/시험성적서 어디에도 없음 → "찾을 수 없습니다"
ask_with_sources("RF9000 가격은 얼마인가요?")


In [ ]:
# 심화 Q2: 경쟁사 비교 — 다른 회사 제품 정보는 문서에 없음
ask_with_sources("삼성 냉장고와 비교하면?")


In [ ]:
# 심화 Q3: AS 전화번호 — 고객 서비스 정보는 문서에 없음
ask_with_sources("이 제품 AS 전화번호는?")


---

## 자유 질문

> 궁금한 질문을 직접 만들어서 테스트해보세요!

In [ ]:
ask_with_sources("") # ← 여기에 질문 입력

In [ ]:
ask_with_sources("") # ← 여기에 질문 입력

In [ ]:
ask_with_sources("") # ← 여기에 질문 입력

---
---
# 실습 완료!

## 오늘 만든 것 정리

| 산출물 | 확인 |
|--------|------|
| 기본 RAG: `qa` 체인으로 문서에 질문 → 답변 + 출처 | |
| 커스텀 프롬프트: 규칙 3가지 + 나의 규칙 추가 | |
| 출처 표시: `ask_with_sources()` 함수 구현 | |
| "모름" 응답: 문서에 없으면 지어내지 않기 | |

---

### 오후3 (15:30~16:30): 미니퀴즈 + 조별 토론

> **이 노트북을 제출합니다.** (산출물 확인용)